### Use case 3 - Querying data in DynamoDB

Once we have our DynamoDB table loaded with data, it's time to retrieve the data, either a specific record or the ability to perform a query, whether by index or sort key (or both).

In the session where we worked with DynamoDB, we learned that we can query the NoSQL database using a subset of SQL called PartiQL. In the following examples, we will show how to perform operations via the DynamoDB API and through PartiQL.

If we want to retrieve the 2014 movie "Interstellar," we would do:

### Using get_item

In [1]:
import boto3
import os
import json
import pprint

In [2]:
"""
    The credentials are stored into environment variables:
"""

AWS_ACCESS_KEY = os.getenv('AWS_ACCESS_KEY')
AWS_SECRET_KEY = os.getenv('AWS_SECRET_KEY')

AWS_REGION = os.getenv('AWS_REGION')

In [3]:
dynamodb = boto3.resource(
    'dynamodb', 
    region_name= AWS_REGION, 
    aws_access_key_id= AWS_ACCESS_KEY, 
    aws_secret_access_key= AWS_SECRET_KEY
)

table = dynamodb.Table('FilmsData')

title = "Interstellar"
year = 2014

response = table.get_item(
        Key={
            'year': year,
            'title': title
        },
        ProjectionExpression="title, info.plot")
item = response['Item']
print('Result of get_item:\n')
pprint.pprint(item)

Result of get_item:

{'info': {'plot': 'A group of explorers make use of a newly discovered '
                  'wormhole to surpass the limitations on human space travel '
                  'and conquer the vast distances involved in an interstellar '
                  'voyage.'},
 'title': 'Interstellar'}


### Using get_item with exceptions

In [4]:
from botocore.exceptions import ClientError

table = dynamodb.Table('FilmsData')

title = "Interstellar"
year = 2014

# Retrieve a movie
print("------ Data for Interstellar")
try:
    response = table.get_item(
        Key={
            'year': year,
            'title': title
        },
        ProjectionExpression="title, info.plot"
    )
except ClientError as e:
    print(e.response['Error']['Message'])
else:
    item = response['Item']
    print(item)

------ Data for Interstellar
{'info': {'plot': 'A group of explorers make use of a newly discovered wormhole to surpass the limitations on human space travel and conquer the vast distances involved in an interstellar voyage.'}, 'title': 'Interstellar'}


### Using PartiQL

In [5]:
clientDDB = boto3.client(
    'dynamodb', 
    region_name= AWS_REGION, 
    aws_access_key_id= AWS_ACCESS_KEY, 
    aws_secret_access_key= AWS_SECRET_KEY
)

# Retrieve a movie with PartiQL
print("------ Data for Interstellar using PartiQL")
resp = clientDDB.execute_statement(
    Statement="SELECT title, info.plot FROM FilmsData WHERE year = 2014 and title='Interstellar'")
item = resp['Items'][0]
print(item)

title = "Interstellar"
year = 2014
# Retrieve a movie with PartiQL using specified parameters using "?"
print("------ Data for Interstellar using PartiQL with parameters")
resp = clientDDB.execute_statement(Statement='SELECT * FROM FilmsData WHERE year = ? AND title = ?',
                                    Parameters=[{'N': str(year)}, {'S': title}])
item = resp['Items'][0]
print(item)

------ Data for Interstellar using PartiQL
{'title': {'S': 'Interstellar'}, 'plot': {'S': 'A group of explorers make use of a newly discovered wormhole to surpass the limitations on human space travel and conquer the vast distances involved in an interstellar voyage.'}}
------ Data for Interstellar using PartiQL with parameters
{'year': {'N': '2014'}, 'info': {'M': {'rank': {'N': '493'}, 'actors': {'L': [{'S': 'Anne Hathaway'}, {'S': 'Matt Damon'}, {'S': 'Jessica Chastain'}]}, 'release_date': {'S': '2014-11-05T00:00:00Z'}, 'plot': {'S': 'A group of explorers make use of a newly discovered wormhole to surpass the limitations on human space travel and conquer the vast distances involved in an interstellar voyage.'}, 'genres': {'L': [{'S': 'Sci-Fi'}]}, 'directors': {'L': [{'S': 'Christopher Nolan'}]}}}, 'title': {'S': 'Interstellar'}}


In the case of <a href="https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/dynamodb/client/execute_statement.html#">using PartiQL queries with the execute_statement function</a>, it's worth noting that:

* The queries are case sensitive.
* Parameters are indicated using `?`
* he contents of the parameters are indicated using a list with one dictionary per parameter where the key is the parameter type, and the value is the data to pass (the data is always passed as a `string`)
* The queries always return a dictionary with a property called `Items` that contains the returned results.

It's worth noting that the structure of the results from querying DynamoDB using the API (which respects the structure defined in the database) is different from the results obtained using PartiQL (which creates an attribute for each retrieved column whose value contains the data type).

### We can also perform other types of queries:

#### Movies from 2016 using query

In [6]:
from boto3.dynamodb.conditions import Key

dynamodb = boto3.resource(
    'dynamodb', 
    region_name= AWS_REGION, 
    aws_access_key_id= AWS_ACCESS_KEY, 
    aws_secret_access_key= AWS_SECRET_KEY
)

# Using query
table = dynamodb.Table('FilmsData')
print("----- Movies from 2016")
resp = table.query(KeyConditionExpression=Key('year').eq(2016))
for i in resp['Items']:
    print(i['year'], ":", i['title'])

# Using PartiQL
print("----- Movies from 2016 using PartiQL")
clientDDB = boto3.client(
    'dynamodb', 
    region_name= AWS_REGION, 
    aws_access_key_id= AWS_ACCESS_KEY, 
    aws_secret_access_key= AWS_SECRET_KEY
)
resp = clientDDB.execute_statement(
    Statement="SELECT title, year FROM FilmsData WHERE year = 2016")
for i in resp['Items']:
    print(i['year']['N'], ":", i['title']['S'])

----- Movies from 2016
2016 : Avatar 2
2016 : Pirates of the Caribbean: Dead Men Tell No Tales
2016 : The Flash
----- Movies from 2016 using PartiQL
2016 : Avatar 2
2016 : Pirates of the Caribbean: Dead Men Tell No Tales
2016 : The Flash


#### Movies whose title is between A and L.

In [8]:
import decimal


class DecimalEncoder(json.JSONEncoder):
    def default(self, o):
        if isinstance(o, decimal.Decimal):
            if o % 1 > 0:
                return float(o)
            else:
                return int(o)
        return super(DecimalEncoder, self).default(o)

year2016 = 2016
initialChar = "A"
finalChar = "F"

tabla = dynamodb.Table('FilmsData')

print("-----Movies whose title is between A and L.")
resp = tabla.query(
    ProjectionExpression="#yr, title, info.genres, info.actors[0]",
    # "year" is a reserved keyword in DynamoDB, so when using it in a query or 
    # PartiQL statement, we need to create an alias for it.
    ExpressionAttributeNames={"#yr": "year"},
    KeyConditionExpression=Key('year').eq(year2016) & Key('title').between(initialChar, finalChar)
)
for i in resp['Items']:
    print(i)
    # {'info': {'actors': ['Zoe Saldana'], 'genres': ['Action', 'Adventure', 'Fantasy', 'Sci-Fi']}, 'year': Decimal('2016'), 'title': 'Avatar 2'}
    # Transform the numeric values from Decimal to Number
    print(json.dumps(i, cls=DecimalEncoder))
    # {"info": {"actors": ["Zoe Saldana"], "genres": ["Action", "Adventure", "Fantasy", "Sci-Fi"]}, "year": 2016, "title": "Avatar 2"}
    for genre in i['info']['genres']:
        print(genre)

print("-----Movies whose title is between A and L using PartiQL")
query = 'SELECT year, title, info.genres, info.actors[0] FROM FilmsData WHERE year = ? AND title between ? and ?'
resp = clientDDB.execute_statement(Statement=query,
                                Parameters=[{'N': str(year2016)}, {'S': initialChar}, {'S': finalChar}])
for i in resp['Items']:
    print(i)
    # [{'year': {'N': '2016'}, 'title': {'S': 'Avatar 2'}, 'actors[0]': {'S': 'Zoe Saldana'}, 'genres': {'L': [{'S': 'Action'}, {'S': 'Adventure'}, {'S': 'Fantasy'}, {'S': 'Sci-Fi'}]}}]
    for genre in i['genres']['L']:
        print(genre['S'])

-----Movies whose title is between A and L.
{'info': {'actors': ['Zoe Saldana'], 'genres': ['Action', 'Adventure', 'Fantasy', 'Sci-Fi']}, 'year': Decimal('2016'), 'title': 'Avatar 2'}
{"info": {"actors": ["Zoe Saldana"], "genres": ["Action", "Adventure", "Fantasy", "Sci-Fi"]}, "year": 2016, "title": "Avatar 2"}
Action
Adventure
Fantasy
Sci-Fi
-----Movies whose title is between A and L using PartiQL
{'year': {'N': '2016'}, 'title': {'S': 'Avatar 2'}, 'actors[0]': {'S': 'Zoe Saldana'}, 'genres': {'L': [{'S': 'Action'}, {'S': 'Adventure'}, {'S': 'Fantasy'}, {'S': 'Sci-Fi'}]}}
Action
Adventure
Fantasy
Sci-Fi


The `DecimalEncoder` class is used to transform the Decimal fields that DynamoDB uses to store numerical content into integer or float type as needed.

### Full Scan

When using <i>PartiQL</i>, if we do not specify an expression in the condition that searches for one of the keys, a full scan will be performed on the entire table, which can imply unexpected costs both economically and in terms of provisioned performance.

The `scan` method reads each item in the table and returns all the data from the table. An optional `filter_expression` can be passed to only return items that meet the criteria. However, the filtering is applied after scanning the entire table.

In [9]:
table = dynamodb.Table('FilmsData')

print("-----Movies with an outstanding rating using full scan.")
# Scan and filtering
fe = Key('info.rating').gte(9)
pe = "#yr, title, info.rating"
ean = {"#yr": "year"}
resp = table.scan(
    FilterExpression=fe,
    ProjectionExpression=pe,
    ExpressionAttributeNames=ean
)
for i in resp['Items']:
    print(json.dumps(i, cls=DecimalEncoder))

-----Movies with an outstanding rating using full scan.
{"info": {"rating": 9}, "year": 1994, "title": "Pulp Fiction"}
{"info": {"rating": 9.3}, "year": 1994, "title": "The Shawshank Redemption"}


#### Full scan using PartiQL

In [10]:
print("-----Movies from 2016 using PartiQL")
resp = clientDDB.execute_statement(
    Statement="SELECT title, year, info.rating FROM FilmsData WHERE info.rating >= 9")
for i in resp['Items']:
    print(i['year']['N'], ":", i['title']['S'])

-----Movies from 2016 using PartiQL
1994 : Pulp Fiction
1994 : The Shawshank Redemption
